# Build spectral library for identified structures isolated from cystobacter

## Import library

In [ ]:
# Install dependency
```bash
pip install ms_entropy,matchms,SpectalEntropy,spectrum-utils,FPSim2
```

In [273]:
import os,json,ujson,ms_entropy,matchms,json,spectral_entropy,sys,time
sys.path.append('../')
import pandas as pd
import numpy as np
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
from tqdm import tqdm, trange
from ms_entropy import FlashEntropySearch
from my_packages import cheminfo_tools
import spectrum_utils.spectrum as sus
from my_packages.peaktools import cosine
from FPSim2.io import create_db_file
from FPSim2 import FPSim2Engine

## Load GNPS spectral library


In [1]:
GNPS_CLEAN_FILE = './cystoscape/ALL_GNPS_cleaned.mgf'
# This file is about 2 GB and should be downloaded at https://external.gnps2.org/gnpslibrary

In [4]:
t = time.time() 
GNPS_CLAEN_INFO = list(load_from_mgf(GNPS_CLEAN_FILE))
print(f'Finished in {(time.time() - t) / 60:.2f} min')

Finished in 9.39 min


In [43]:
IDX2SMILE = []
for idx, SPECTRUM in tqdm(enumerate(GNPS_CLAEN_INFO),total = len(GNPS_CLAEN_INFO)):
    IONMODE = SPECTRUM.metadata['ionmode']
    if IONMODE == 'positive':
        try:
            SMILE = SPECTRUM.metadata['smiles']
            IDX2SMILE.append([SMILE,idx]) 
        except:pass

100%|████████████████████████████████| 913576/913576 [00:11<00:00, 79115.92it/s]


In [344]:
# Generate chemical chemical fingerprint database
t = time.time()
create_db_file(
    mols_source=IDX2SMILE, 
    filename='./cystobacter/fp_db.h5',  # output
    mol_format='smiles', # required
    fp_type='Morgan',
    fp_params={'radius': 2, 'fpSize': 1024}
)
print(f'Finished in {(time.time() - t) / 60:.2f} min')

Finished in 3.21 min


## Build library for *Cystobacter*


In [25]:
# Load list of collected metabolites from Cystobacter
XLSX_FILE = './cystobacter/CysCompounds.xlsx'
CYS_DF = pd.read_excel(XLSX_FILE,index_col = None) # npaid,compound_smiles

In [359]:
# Example
fp_filename = './cystobacter/fp_db.h5'
fpe = FPSim2Engine(fp_filename)

query = 'CC(=O)Oc1ccccc1C(=O)O'
# results = fpe.top_k(query, k=10, threshold=0.7, metric='tanimoto', n_workers=1)
results = fpe.similarity(query, threshold=0.8, metric='tanimoto', n_workers=1)
fpidx_list = [x[0] for x in results]
fpsim_list = [x[1] for x in results]

In [383]:
# Retrieve CCMSLIB_ID of similar structures with Cystobacter's metabolites
hit_dict = {}
for idx in trange(len(CYS_DF)):
    NPAID = CYS_DF.loc[idx,'npaid']
    CYSSMILE = CYS_DF.loc[idx,'compound_smiles']
    results = fpe.top_k(CYSSMILE, k=5,threshold=0.5, metric='tanimoto', n_workers=1) # [(CCMSLIBidx,similarity1),(),...]
    fpidx_list = [x[0] for x in results]
    fpsim_list = [x[1] for x in results]
    if fpidx_list:
        hit_dict[idx] = {'npaid':NPAID,'np_smile':CYSSMILE,'CCMSLIB_idx':fpidx_list,'sim_list':fpsim_list}

100%|███████████████████████████████████████████| 58/58 [00:01<00:00, 34.44it/s]


In [385]:
# Dereplication and save non-redundant LIBIDXs
NR_LIBIDX =[]
for key, value in hit_dict.items():
    ids = value['CCMSLIB_idx']
    NR_LIBIDX.extend(ids)
NR_LIBIDX = list(set(NR_LIBIDX))
for libidx in NR_LIBIDX:
    save_as_mgf(GNPS_CLAEN_INFO[libidx], "./cystobacter/Lib4Cys.mgf")
    

## Load cbfe23 mgf

In [105]:
# Query FS formatting
CF_MGF = "./cystobacter/cbfe23.mgf" # the same mgf file depict figure 4A
SPECTRA = load_from_mgf(CF_MGF)
FS_SPECTRA = []
for SPECTRUM in SPECTRA:
    FID = SPECTRUM.metadata['feature_id']
    PM = SPECTRUM.metadata['precursor_mz']
    peaks = np.column_stack((SPECTRUM.mz,SPECTRUM.intensities))
    FS_SPECTRA.append(
        {
            "id": FID,
            "precursor_mz": PM,
            "peaks": peaks
        }
    )

sort = FlashEntropySearch()
FS_SPECTRA = sort.build_index(FS_SPECTRA) # Pre-clean and sorted flash spectra

In [ ]:
# Flash formatting reference library
LIB_MGF = "./cystobacter/Lib4Cys.mgf"
LIB_SPECTRA = load_from_mgf(LIB_MGF)


FS_LIB_SPECTRA = []
for SPECTRUM in LIB_SPECTRA:
    FID = SPECTRUM.metadata['spectrum_id']
    PM = SPECTRUM.metadata['precursor_mz']
    SMILES = SPECTRUM.metadata['smiles']
    peaks = np.column_stack((SPECTRUM.mz,SPECTRUM.intensities))
    FS_LIB_SPECTRA.append(
        {
            "id": FID,
            "precursor_mz": PM,
            "peaks": peaks,
            "SMILES": SMILES
        }
    )

lib_search1 = FlashEntropySearch()
FS_LIB_SPECTRA = lib_search1.build_index(FS_LIB_SPECTRA) # Pre-clean and sorted flash spectra

In [397]:
for s in FS_LIB_SPECTRA:
    print(s['SMILES'])

CC(C)CC(=O)NCC(=O)O
CC(C)CC(=O)NCC(=O)O
CC(C)CC(=O)NCC(=O)O
CC(C)CC(=O)NCC(=O)O
CC(C)CC(=O)NCC(=O)O
CCCCCC(=O)NCC(=O)O
CCCCCC(=O)NCC(=O)O
CCCCCC(=O)NCC(=O)O
CCCCCC(=O)NCC(=O)O
CCCCCC(=O)NCC(=O)O
CC(=O)NCCc1ccc(O)cc1
CC(=O)NCCc1ccc(O)cc1
CC(=O)NCCc1ccc(O)cc1
CC(=O)NCCc1ccc(O)cc1
CC(=O)NCCc1ccc(O)cc1
CC(C)CC(N)C(=O)NC(Cc1ccccc1)C(=O)O
CC(C)CC(N)C(=O)NC(Cc1ccccc1)C(=O)O
CC(C)CC(N)C(=O)NC(Cc1ccccc1)C(=O)O
CC(C)CC(N)C(=O)NC(Cc1ccccc1)C(=O)O
CC(C)CC(N)C(=O)NC(Cc1ccccc1)C(=O)O
NC(Cc1c[nH]c2ccccc12)C(=O)NC(Cc1ccccc1)C(=O)O
NC(Cc1c[nH]c2ccccc12)C(=O)NC(Cc1ccccc1)C(=O)O
NC(Cc1ccccc1)C(=O)NC(Cc1c[nH]c2ccccc12)C(=O)O
NC(Cc1c[nH]c2ccccc12)C(=O)NC(Cc1ccccc1)C(=O)O
NC(Cc1c[nH]c2ccccc12)C(=O)NC(Cc1ccccc1)C(=O)O
CC(C=CC=CC=CC=C(C)C(=O)NC(C)CO)=CC(C)C(O)C(C)=CC(C)C
CC(C=CC=CC=CC=C(C)C(=O)NC(C)CO)=CC(C)C(O)C(C)=CC(C)C
O=C(O)CCC(NCC1(O)OCC(O)C(O)C1O)C(=O)NC(Cc1ccccc1)C(=O)O
CCC(C)C(NC(=O)C1CCCCN1C)C(=O)N(COC(=O)CC(C)C)C(CC(OC(C)=O)c1nc(C(=O)NC(Cc2ccc(O)cc2)CC(C)C(=O)O)cs1)C(C)C
COc1c(NC(=O)c2ccc(NC(=O)c3c

In [388]:
# Search
SEARCH_MATRIX = {} # {FID:{search_matrix}}
for FIDX, SPECTRUM in enumerate(FS_SPECTRA):
    FID = SPECTRUM['id']
    PM = SPECTRUM['precursor_mz']
    PEAKs = SPECTRUM['peaks']
    SIM_MATRIX = lib_search1.search(precursor_mz = PM, peaks = PEAKs)

    SEARCH_MATRIX[FIDX] = SIM_MATRIX

In [398]:
# Filter results
Search_res = {}
for FIDX, SIM_MATRIX in SEARCH_MATRIX.items():
    NR_HITS_LIST = []
    for search_mode, sim_arry in SIM_MATRIX.items():
        hits = np.where(sim_arry >= 0.5)
        NR_HITS_LIST.extend(hits)
    NR_HITS_LIST = np.unique(np.concatenate(NR_HITS_LIST))[:5] # dereplicating and keep top 5 matches
    Search_res[FIDX] = NR_HITS_LIST

In [399]:
# View matched pairs
for key, value in Search_res.items():
    if value.any():
        FID = FS_SPECTRA[key]['id']
        print(key,FID,Search_res[key])
        

171 175 [28]
172 287 [28]
173 68 [28]
177 177 [28]
178 291 [28]
179 70 [28]
180 179 [28]
181 293 [28]
182 294 [28]
183 72 [28]


In [289]:
# View specific query spectra
FS_SPECTRA[12]

{'id': '5',
 'precursor_mz': 211.094,
 'peaks': array([[1.6307291e+02, 8.8571429e-01],
        [1.6407471e+02, 8.2095236e-02],
        [1.8108340e+02, 1.7142856e-02],
        [1.9406174e+02, 1.5047619e-02]], dtype=float32)}

In [391]:
# View specific library spectra
FS_LIB_SPECTRA[28]

{'id': 'CCMSLIB00000478582',
 'precursor_mz': 844.453,
 'peaks': array([[2.11180740e+02, 1.13930985e-01],
        [2.12183899e+02, 1.59014687e-02],
        [2.21074326e+02, 6.24685781e-03],
        [2.38100998e+02, 4.11649793e-03],
        [2.39087738e+02, 2.76081450e-03],
        [2.39175980e+02, 2.37895682e-01],
        [2.40178696e+02, 4.39628810e-02],
        [2.41181366e+02, 4.37197648e-03],
        [3.17095306e+02, 5.03539620e-03],
        [3.43111481e+02, 2.76905578e-03],
        [3.61121582e+02, 5.31147746e-03],
        [3.97157837e+02, 3.96815594e-03],
        [3.99173523e+02, 5.05599938e-03],
        [4.15168518e+02, 3.79508990e-03],
        [4.17183899e+02, 3.17287655e-03],
        [4.26184662e+02, 6.62842765e-02],
        [4.27187439e+02, 1.84397697e-02],
        [4.28184784e+02, 4.29780548e-03],
        [4.44195282e+02, 8.35785121e-02],
        [4.45198151e+02, 2.39655189e-02],
        [4.46195770e+02, 5.72765991e-03],
        [4.62205658e+02, 7.43359607e-03],
        [4.7